# Confidence Intervals
<hr>

A confidence interval is a range is which we are confident to a certain degree that a specific population parameter will fall. 

For example, according to [glassdoor](https://www.glassdoor.com/Salaries/data-scientist-salary-SRCH_KO0,14.htm), as of April 7th 2020, the mean salary for data scientists is \\$113,309. Now obviously glassdoor hasn't polled every working data scientist that exists to get to this number, meaning \\$113,309 is an estimate of the true mean.

So if we were to construct a 95% CI for this average and (just making up the numbers) got \\$100,000 - \\$115,000. This means that if we were to poll many times the average salary of __random__ samples of data scientists, than in 95% of those cases we would get an average salary that falls in that range.

In [2]:
import pandas as pd
import numpy as np
# we'll use both a normal distribution and t distribution
from scipy.stats import norm, t

## Population variance known

__Note:__ In this notebook every $\sigma$ represents a population standard deviation while $s$ represents a sample standard deviation.